<div id="singlestore-header" style="display: flex; background-color: rgba(209, 153, 255, 0.25); padding: 5px;">
    <div id="icon-image" style="width: 90px; height: 90px;">
        <img width="100%" height="100%" src="https://raw.githubusercontent.com/singlestore-labs/spaces-notebooks/master/common/images/header-icons/notes.png" />
    </div>
    <div id="text" style="padding: 5px; margin-left: 10px;">
        <div id="badge" style="display: inline-block; background-color: rgba(0, 0, 0, 0.15); border-radius: 4px; padding: 4px 8px; align-items: center; margin-top: 6px; margin-bottom: -2px; font-size: 80%">SingleStore Notebooks</div>
        <h1 style="font-weight: 500; margin: 8px 0 0 4px;">Basic Query Examples</h1>
    </div>
</div>

<div class="alert alert-block alert-warning">
    <b class="fa fa-solid fa-exclamation-circle"></b>
    <div>
        <p><b>Note</b></p>
        <p>This notebook can be run on a Free Starter Workspace. You can also use your existing Standard or Enterprise workspace with this Notebook.</p>
    </div>
</div>

<table style="border: 0; border-spacing: 0; width: 100%; background-color: #03010D"><tr>
    <td style="padding: 0; margin: 0; background-color: #03010D; width: 33%; text-align: center"><img src="https://raw.githubusercontent.com/singlestore-labs/spaces-notebooks/master/common/images/singlestore-logo-vertical.png" style="height: 200px;"/></td>
    <td style="padding: 0; margin: 0; width: 66%; background-color: #03010D; text-align: right"><img src="https://raw.githubusercontent.com/singlestore-labs/spaces-notebooks/master/common/images/singlestore-jupyter.png" style="height: 250px"/></td>
</tr></table>

This notebook demonstrates how to run queries on SingleStore through a series of examples. For simplicity, the sample data in these examples is limited to 10 rows or less per table, which is much smaller than typical workloads. The examples cover various database operations, including index scans, full-table scans, joins, and aggregations.

## 1. Create a Workspace
To create a workspace, refer to the [Creating and Using Workspaces](https://docs.singlestore.com/cloud/getting-started-with-singlestore-helios/about-workspaces/creating-and-using-workspaces/).

## 2. Create the Database
Select the workspace in your notebook, and create a database.

In [16]:
%%sql
CREATE DATABASE memsql_example;
USE memsql_example;

++
||
++
++

## 3. Create the tables
Create the tables named departments, employees, and salaries.

In [18]:
%%sql
CREATE TABLE departments (
  id int,
  name varchar(255),
  PRIMARY KEY (id)
);

CREATE TABLE employees (
  id int,
  deptId int,
  managerId int,
  name varchar(255),
  hireDate date,
  state char(2),
  PRIMARY KEY (id)
);

CREATE TABLE salaries (
  employeeId int,
  salary int,
  PRIMARY KEY (employeeId)
);

++
||
++
++

## 4. Populate the tables
Insert the data into the tables.

In [19]:
%%sql
INSERT INTO departments (id, name) VALUES
  (1, 'Marketing'), (2, 'Finance'), (3, 'Sales'), (4, 'Customer Service');

INSERT INTO employees (id, deptId, managerId, name, hireDate, state) VALUES
  (1, 2, NULL, "Karly Steele", "2011-08-25", "NY"),
  (2, 1, 1, "Rhona Nichols", "2008-09-11", "TX"),
  (3, 4, 2, "Hedda Kent", "2005-10-27", "TX"),
  (4, 2, 1, "Orli Strong", "2001-07-01", "NY"),
  (5, 1, 1, "Leonard Haynes", "2011-05-30", "MS"),
  (6, 1, 5, "Colette Payne", "2002-10-22", "MS"),
  (7, 3, 4, "Cooper Hatfield", "2010-08-19", "NY"),
  (8, 2, 4, "Timothy Battle", "2001-01-21", "NY"),
  (9, 3, 1, "Doris Munoz", "2008-10-22", "NY"),
  (10, 4, 2, "Alea Wiggins", "2007-08-21", "TX");

INSERT INTO salaries (employeeId, salary) VALUES
  (1, 885219), (2, 451519), (3, 288905), (4, 904312), (5, 919124),
  (6, 101538), (7, 355077), (8, 900436), (9, 41557), (10, 556263);

++
||
++
++

## 5. Let's Query!

### The SELECT statement
Ask how many rows are in the employees table.

In [20]:
%%sql
SELECT COUNT(*) from employees;

COUNT(*)
10


### The ORDER BY clause
List the ID and the name of each employee.

In [21]:
%%sql
SELECT id, name FROM employees ORDER BY id;

id,name
1,Karly Steele
2,Rhona Nichols
3,Hedda Kent
4,Orli Strong
5,Leonard Haynes
6,Colette Payne
7,Cooper Hatfield
8,Timothy Battle
9,Doris Munoz
10,Alea Wiggins


### The WHERE clause
List the employees that work in Texas.

In [22]:
%%sql
SELECT id, name FROM employees WHERE state = 'TX' ORDER BY id;

id,name
2,Rhona Nichols
3,Hedda Kent
10,Alea Wiggins


You can use the same above query by replacing **state = 'NY'** to list the employees that work in New York.

In [23]:
%%sql
SELECT id, name FROM employees WHERE state = 'NY' ORDER BY id;

id,name
1,Karly Steele
4,Orli Strong
7,Cooper Hatfield
8,Timothy Battle
9,Doris Munoz


List the employees hired before 2002.

In [24]:
%%sql
SELECT id, name, hireDate
  FROM employees
  WHERE hireDate < '2002-01-01'
  ORDER BY id;

id,name,hireDate
4,Orli Strong,2001-07-01
8,Timothy Battle,2001-01-21


List employees and their departments.

In [25]:
%%sql
SELECT e.name, d.name department FROM
  employees e, departments d
  WHERE e.deptId = d.id
  ORDER BY name;

name,department
Alea Wiggins,Customer Service
Colette Payne,Marketing
Cooper Hatfield,Sales
Doris Munoz,Sales
Hedda Kent,Customer Service
Karly Steele,Finance
Leonard Haynes,Marketing
Orli Strong,Finance
Rhona Nichols,Marketing
Timothy Battle,Finance


### The COUNT function
List the number of employees in each state.

In [26]:
%%sql
SELECT state, COUNT(*)
  from employees
  group by state
  ORDER BY state;

state,COUNT(*)
MS,2
NY,5
TX,3


### The MAX function
Highest salary amongst all employees.

In [27]:
%%sql
SELECT MAX(salary) FROM salaries;

MAX(salary)
919124


### The Subqueries
Employee with the highest salary.

In [28]:
%%sql
SELECT e.name, s.salary
  FROM employees e, salaries s
  WHERE e.id = s.employeeId and
    s.salary = (SELECT MAX(salary) FROM salaries);

name,salary
Leonard Haynes,919124


### The AVG function
Average salary of employees in each state.

In [29]:
%%sql
SELECT e.state, AVG(salary)
  FROM employees e
  JOIN salaries s on e.id = s.employeeId
  GROUP BY e.state
  ORDER BY e.state;

state,AVG(salary)
MS,510331.0000
NY,617320.2000
TX,432229.0000


### The IN operator
List of managers.

In [30]:
%%sql
SELECT name
  FROM employees
  WHERE id IN (SELECT managerId FROM employees)
  ORDER BY name;

name
Karly Steele
Leonard Haynes
Orli Strong
Rhona Nichols


### The NOT IN operator
List of non-managers.

In [31]:
%%sql
SELECT name
  FROM employees
  WHERE id NOT IN (SELECT managerId FROM employees)
  ORDER BY name;

name
Alea Wiggins
Colette Payne
Cooper Hatfield
Doris Munoz
Hedda Kent
Timothy Battle


### The Joins
Number of employees reporting to each manager.

In [32]:
%%sql
SELECT m.name, COUNT(*) count
  FROM employees m
  JOIN employees e ON m.id = e.managerId
  GROUP BY m.id
  ORDER BY count DESC;

name,count
Karly Steele,4
Rhona Nichols,2
Orli Strong,2
Leonard Haynes,1


Number of employees reporting to each employee.

In [33]:
%%sql
SELECT m.name, COUNT(e.id) count
  FROM employees m
  LEFT JOIN employees e ON m.id = e.managerId
  GROUP BY m.id
  ORDER BY count desc;

name,count
Karly Steele,4
Rhona Nichols,2
Orli Strong,2
Leonard Haynes,1
Alea Wiggins,0
Timothy Battle,0
Cooper Hatfield,0
Doris Munoz,0
Hedda Kent,0
Colette Payne,0


Manager of each employee.

In [34]:
%%sql
SELECT e.name employee_name, m.name manager_name
  FROM employees e
  LEFT JOIN employees m ON e.managerId = m.id
  ORDER BY manager_name;

employee_name,manager_name
Karly Steele,None
Rhona Nichols,Karly Steele
Orli Strong,Karly Steele
Leonard Haynes,Karly Steele
Doris Munoz,Karly Steele
Colette Payne,Leonard Haynes
Timothy Battle,Orli Strong
Cooper Hatfield,Orli Strong
Alea Wiggins,Rhona Nichols
Hedda Kent,Rhona Nichols


Total salary of the employees reporting to each manager.

In [35]:
%%sql
SELECT m.name, sum(salary)
  FROM employees m
  JOIN employees e ON m.id = e.managerId
  JOIN salaries s ON s.employeeId = e.id
  GROUP BY m.id
  ORDER BY SUM(salary) DESC;

name,sum(salary)
Karly Steele,2316512
Orli Strong,1255513
Rhona Nichols,845168
Leonard Haynes,101538


Employees in the finance department earning more than their manager.

In [ ]:
%%sql
%%sql
SELECT e.name employee_name, se.salary employee_salary, m.name manager_name, sm.salary manager_salary
FROM employees e
JOIN salaries se ON e.id = se.employeeId
JOIN employees m ON m.id = e.managerId
JOIN salaries sm ON sm.employeeId = m.id
JOIN departments d ON d.id = e.deptId
WHERE d.name = 'Finance'
  AND sm.salary < se.salary
ORDER BY employee_salary, manager_salary;

**Learn more about SingleStore notebooks [here](https://docs.singlestore.com/managed-service/en/developer-resources/notebooks.html).**